In [1]:
import os
import onnxruntime as ort

print("ORT:", ort.__version__)
print("Device:", ort.get_device())
print("Providers:", ort.get_available_providers())


ORT: 1.23.2
Device: GPU
Providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [5]:
import time, numpy as np
from fastembed import TextEmbedding

EMB_MODEL = "BAAI/bge-small-en-v1.5"

embedder = TextEmbedding(
    model_name=EMB_MODEL,
    providers=["TensorrtExecutionProvider", "CUDAExecutionProvider", "CPUExecutionProvider"],
)

texts = ["hello world", "gpu test"] * 2048  # 4096 chunks to load the GPU
t0 = time.time()
vecs = list(embedder.embed(texts, batch_size=1024, parallel=None))
dt = time.time() - t0

arr = np.vstack(vecs).astype("float32")
print(arr.shape, "in", f"{dt:.2f}s  →  {arr.shape[0]/dt:.0f} chunks/sec")


*************** EP Error ***************
EP Error E:\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:560 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************
(4096, 384) in 21.09s  →  194 chunks/sec


In [6]:
import time, numpy as np
from fastembed import TextEmbedding

EMB_MODEL = "BAAI/bge-small-en-v1.5"

embedder = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],  # drop TensorRT
)

texts = ["hello world", "gpu test"] * 2048  # 4096 chunks to load the GPU
t0 = time.time()
vecs = list(embedder.embed(texts, batch_size=1024, parallel=None))
dt = time.time() - t0

arr = np.vstack(vecs).astype("float32")
print(arr.shape, "in", f"{dt:.2f}s  →  {arr.shape[0]/dt:.0f} chunks/sec")


(4096, 384) in 21.14s  →  194 chunks/sec


In [1]:
import onnxruntime as ort
print("ORT:", ort.__version__)
print("Providers:", ort.get_available_providers())


ORT: 1.23.2
Providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [2]:
import time, numpy as np
from fastembed import TextEmbedding

embedder = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],  # no TensorRT yet
)

texts = ["hello world", "gpu test"] * 2048
t0 = time.time()
vecs = list(embedder.embed(texts, batch_size=1024, parallel=None))
dt = time.time() - t0
arr = np.vstack(vecs).astype("float32")
print(arr.shape, "in", f"{dt:.2f}s  →  {arr.shape[0]/dt:.0f} chunks/sec")


C:\Users\Lenovo\Projects\QnA on YouTube Video - RAG\.venv\Lib\site-packages\fastembed\common\onnx_model.py:109: RuntimeWarning: Attempt to set CUDAExecutionProvider failed. Current providers: ['CPUExecutionProvider'].If you are using CUDA 12.x, install onnxruntime-gpu via `pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/`
  warnings.warn(


(4096, 384) in 21.48s  →  191 chunks/sec


In [3]:
import os
os.environ["ORT_LOG_SEVERITY_LEVEL"]="0"
os.environ["ORT_CUDA_VERBOSE_LOGGING"]="1"
from fastembed import TextEmbedding
_ = TextEmbedding(model_name="BAAI/bge-small-en-v1.5",
                  providers=["CUDAExecutionProvider","CPUExecutionProvider"])


In [6]:
_

In [2]:
import time, torch, numpy as np
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

texts = ["hello world","gpu test"]*2048
t0=time.time()
with torch.inference_mode():
    arr = model.encode(texts, batch_size=1024, convert_to_numpy=True, normalize_embeddings=True)
dt=time.time()-t0
print(arr.shape,"in",f"{dt:.2f}s → {arr.shape[0]/dt:.0f} chunks/sec")


cuda
(4096, 384) in 0.48s → 8550 chunks/sec
